### 参考文献
萩原淳一郎, 瓜生真也, 牧山幸史. 基礎からわかる時系列分析―Rで実践するカルマンフィルタ・MCMC・粒子フィルタ―. Data Science Library  
カルマンフィルタ ―Rを使った時系列予測と状態空間モデル― (統計学One Point 2)  
https://www.avelio.co.jp/math/wordpress/?p=605

- - - 

# 状態空間モデル(Space State Model: SSM)

* 推定対象は値ではなく確率分布
* 直接観測されるデータに加えて、直感的には観測されない潜在的な確率変数(=状態)を導入。
* 状態はマルコフ性(前時点のみと関連がある)を仮定する。
* ある時点の観測値はその時点の状態によってのみ決まる。

## 0. 定義

方程式によって表現すると

$$
\left\{
\begin{array}{ll}
\mathbf{x}_{t} = f(\mathbf{x}_{t-1}, \mathbf{w}_{t}) & \dots \text{状態方程式orシステム方程式}\\
y_{t} = g(\mathbf{x}_{t}+v_{t}) & \dots 観測方程式
\end{array}
\right.
$$
ただし$f(\cdot), g(\cdot)$は任意の関数、$\mathbf{w}_{t}, v_{t}$はそれぞれ状態雑音、観測雑音と呼ばれるホワイトノイズ。

## 1. 分類

$f(\cdot), g(\cdot)$の種類によってモデルの形状に名前がついている

|  | $\mathbf{w}_{t}, v_{t}$のいずれかが非ガウス分布 | $\mathbf{w}_{t}, v_{t}$がともにガウス分布 |
| - | - | - |
| $f(\cdot), g(\cdot)$のいずれかが非線形 | 非線形・非ガウス型状態空間モデル | 非線形・ガウス型状態空間モデル |
| $f(\cdot), g(\cdot)$がともに線形 | 線形・非ガウス型状態空間モデル | 線形・ガウス型状態空間モデル <br>or 動的線形モデル(Dynamic Linear Model: DLM) |  

特に
### 線形・ガウス型状態空間モデル
$$
\left\{
\begin{array}{ll}
\mathbf{x}_{t} = \mathbf{G}_{t}\mathbf{x}_{t-1}+\mathbf{w}_{t},  & \mathbf{w}_{t} \sim N(\mathbf{0},\mathbf{W}_{t}), \quad
\mathbf{x}_{0} \sim N(\mathbf{m}_{0}, \mathbf{C}_{0})\\
y_{t} = \mathbf{F}_{t}\mathbf{x}_{t}+v_{t},  & v_{t} \sim N(0, V_{t})
\end{array}
\right.
$$
もしくは
$$
\left\{
\begin{array}{ll}
p(\mathbf{x}_{t}|\mathbf{x}_{t-1}) = N(\mathbf{G}_{t}\mathbf{x}_{t-1}, \mathbf{W}_{t})\\
p(\mathbf{x}_{0}) = N(\mathbf{m}_{0}, \mathbf{C}_{0})\\
p(y_{t}|\mathbf{x}_{t}) = N(\mathbf{F}_{t}\mathbf{x}_{t-1}, V_{t})
\end{array}
\right.
$$
ここで、  
$\mathbf{G}_{t}$は$p \times p$の**状態遷移行列**、$\mathbf{F}_{t}$は$1 \times p$の**観測行列**、$\mathbf{W}_{t}$は$p \times p$の状態雑音の共分散行列、$\mathbf{V}_{t}$は観測雑音の分散、
$\mathbf{m}_{0}$は事前分布における$p$次元の平均ベクトル、$\mathbf{C}_{0}$は事前分布における$p \times p$の共分散行列とする。

## 2. 状態の推定
* 平滑化  
  すべてのデータが手に入ったときそのあとに、状態の補正を行う計算。知識発見という意味合いで使われることが多い。
* フィルタリング  
  手に入った観測値を用いて、予測された状態の値を補正すること。
* 予測

## 3. パラメータの推定

SSMのパラメータをまとめて$\mathbf{\theta}$とする

### 3-1. パラメータを確率分布として考えない場合(頻度論的)

#### 最尤推定  
尤度は
$$
\begin{align*}
p(y_{1}, y_{2}, \dots, y_{T};\mathbf{\theta}) &= p(y_{1:T};\mathbf{\theta}) \\
&= p(y_{T}, y_{1:T-1}; \mathbf{\theta}) \\
&= p(y_{T}|y_{1:T-1};\mathbf{\theta})p(y_{1:T-1};\mathbf{\theta}) \\
&= \prod_{t=1}^{T} p(y_{t}|y_{1:t-1};T) \\
\end{align*}
$$
ここで$y_{1:0}$は存在しないので空集合とすると
$p(y_{1}|y_{1:0};\mathbf{\theta}) = p(p_{1}|\emptyset;\mathbf{\theta}) = p(y_{1};\mathbf{\theta})$。  
対数とると
$$
\begin{align*}
l(\mathbf{\theta}) &= \log{p(y_{1}, y_{2}, \dots, y_{T};\mathbf{\theta})} \\
&= \sum_{t=1}^{T} \log{p(y_{t}|y_{1:t-1};T)}
\end{align*}
$$

従って最尤推定値$\mathbf{\hat{\theta}}$は
$$
\begin{align*}
\mathbf{\hat{\theta}} &= \underset{\mathbf{\hat{\theta}}}{\text{argmax }} l(\mathbf{\theta}) \\
&= \underset{\mathbf{\hat{\theta}}}{\text{argmax }} \log{p(y_{1:T}|\mathbf{\theta})}\\
&= \underset{\mathbf{\hat{\theta}}}{\text{argmax }} \sum_{t=1}^{T} \log{p(y_{t}|y_{1:t-1};T)}
\end{align*}
$$

### 3-2. パラメータを確率分布として考える場合(ベイズ的)

#### MAP推定(周辺事後分布を最大化する)

$$
\begin{align*}
\underset{\mathbf{\theta}}{\text{argmax }}p(\mathbf{\theta}|y_{1:T})
&=  \underset{\mathbf{\theta}}{\text{argmax }}\frac{p(y_{1:T}|\mathbf{\theta})p(\mathbf{\theta})}{p(y_{1:T})}  \\
&\propto \underset{\mathbf{\theta}}{\text{argmax }}p(y_{1:T}|\mathbf{\theta})p(\mathbf{\theta})
\end{align*}
$$
対数取ると
$$
\underset{\mathbf{\theta}}{\text{argmax }} \log{p(\mathbf{\theta}|y_{1:T})} \propto \underset{\mathbf{\theta}}{\text{argmax }} \{ \log{p(y_{1:T}|\mathbf{\theta})} + \log{p(\mathbf{\theta})}\}
$$

## 4. 状態を推定するアルゴリズム
ウィナーフィルタ、カルマンフィルタ、 散漫カルマンフィルタ、MCMC、粒子フィルタ

- - -
線形ガウス状態空間モデルを以下のように定義する。  
$t=1, \dots, n$において

$$
\begin{align*}
y_t &= Z_{t}\alpha_{t} + \epsilon_{t}, & \epsilon \sim N(0, H_t) \\
\alpha_{t+1} &= T_{t}\alpha_{t} + R_{t} \eta_{t}, & \eta_{t} \sim N(0, Q_t), \\ 
\alpha_{1} &\sim N(a_{1}, P_{1})
\end{align*}
$$
ただし攪乱項と初期状態の組$\epsilon_1, \dots, \epsilon_{n}, \eta_1, \dots, \eta_{n}, \alpha_{1}$は全てお互いに独立であると仮定する。  
観測ベクトル$y_{t}$の次元を$p_{t}$、状態ベクトル$\alpha_{t}$の次元を$m$とおく。  
また1期先予測を$a_{t} = E(\alpha_{t}|Y_{t-1})$、1期先予測分散を$P_{t} = Var(\alpha_{t}|Y_{t-1})$、1期先予測誤差を$v_{t} = y_{t} - E(y_{t}|Y_{t-1})$、  
1期先予測誤差分散を$F_{t}= Var(v_{t}|Y_{t-1})$、フィルタ化推定値$a_{t|t} = E(\alpha_{t}|Y_{t})$で表す。   
$Z_{t}, H_{t}, T_{t}, R_{t}, Q_{t}$は時間と共に値を変化させることができる。
- - -

### カルマンフィルタ  (Kalman Filter)
多変量正規分布の条件付き平均、条件付き分散の結果

$$
\begin{align*}
E(y_{1}|y_{2}) &= \mu_{1} + \Sigma_{12}\Sigma_{12}^{-1}(\mu_2 - \mu_1) \\
Var(y_{1}|y_{2}) &= \Sigma_{11} - \Sigma_{12} \Sigma_{22}^{-1} \Sigma_{12}'
\end{align*}
$$ 
ただし$E(y_{1}) = \mu_{1}, E(y_{2}) = \mu_{2}, \Sigma_{11} = Var(y_{1}), \Sigma_{12} = \Sigma_{21}' = Cov(y_{1}, y_{2}), \Sigma_{22} = Var(y_{2})$  
を利用して、条件付き分布を1時点ずつ更新して求めていくアルゴリズム

In [22]:
import numpy as np

class KalmanFilter:
    """
    カルマンフィルタ
    
    Args:
        system_matrix      (p_t, m)   : 観測行列Z
        observation_matrix (m, m)     : 状態遷移行列T
        system_cov         (p_t, p_t) : 観測雑音の共分散行列H
        observation_cov    (r, r)     : 状態雑音の共分散行列Q
        coefficient_matrix (m, r)     : 状態雑音の係数行列R
        initial_state_mean (p_t)      : 初期分布の平均ベクトルa_1
        initial_state_cov  (p_t, p_t) : 初期分布の共分散行列P_1
    """
    def __init__(self, 
                 system_matrix,
                 observation_matrix,
                 system_cov,
                 observation_cov,
                 coefficient_matrix,
                 initial_state_mean,
                 initial_state_cov,
                 ): 
        self.system_matrix = np.array(system_matrix)
        self.observation_matrix = np.array(observation_matrix)
        self.system_cov = np.array(system_cov)
        self.observation_cov = np.array(observation_cov)
        self.coefficient_matrix = np.array(coefficient_matrix)
        self.initial_state_mean = np.array(initial_state_mean)
        self.initial_state_cov = np.array(initial_state_cov)

#### フィルタリングと予測  


$$
\begin{align*}
v_{t} &= y_{t} - Z_{t}a_{t}, & F_{t} &= Z_{t}P_{t}Z_{t}' + H_{t}, \\
a_{t|t} &= a_{t} + K_{t}v_{t}, & P_{t|t} &= P_{t} - K_{t}F_{t}K_{t}', \\
a_{t+1} &= T_{t}a_{t|t}, & P_{t+1} &= T_{t}P_{t|t}T_{t}' + R_{t}Q_{t}R_{t}' \\
\end{align*}
$$  
ただし$K_{t} = P_{t}Z_{t}F_{t}^{-1}$とし、これをカルマンフィルタと呼ぶ。

In [23]:
def filter_predict(self,
            current_state_mean, 
            current_state_cov,
            ):
    pred_state_mean = self.system_matrix @ current_state_mean
    pred_state_cov = (
        self.system_matrix
        @ current_state_cov
        @ self.system_matrix.T
        + self.coefficient_matrix
        @ self.observation_cov
        @ self.coefficient_matrix.T
    )
    return (pred_state_mean, pred_state_cov)


def filter_update(self, 
                  predicted_state_mean, 
                  predicted_state_cov,
                  observation
                  ):
    error_mean = observation - self.system_matrix @ predicted_state_mean
    error_cov = (
        self.system_matrix
        @ predicted_state_cov
        @ self.system_matrix.T
        + self.system_cov
    )
    kalman_gain = (
        predicted_state_cov
        @ self.observation_matrix
        @ error_cov.I
    )
    
    filtered_state_mean = (
        predicted_state_mean
        + kalman_gain 
        @ error_mean
    )
    
    filtered_state_cov = (
        predicted_state_cov
        - kalman_gain
        @ error_cov
        @ kalman_gain.T
    )
    
    return (filtered_state_mean, filtered_state_cov)

def filter(self, observations):
    observations = np.array(observations)
    n_timesteps = len(observations)
    n_dim_state = len(self.initial_state_mean)
    
    predicted_state_means = np.zeros((n_timesteps, n_dim_state))
    predicted_state_covs= np.zeros((n_timesteps, n_dim_state, n_dim_state))
    filtered_state_means = np.zeros((n_timesteps, n_dim_state))
    filtered_state_covs= np.zeros((n_timesteps, n_dim_state, n_dim_state))
    
    for t in range(n_timesteps):
        if t == 0:
            predicted_state_means[t] = self.initial_state_mean
            predicted_state_covs[t] = self.initial_state_cov
        else:
            predicted_state_means[t], predicted_state_covs[t] = self.filter_predict(
                filtered_state_means[t-1],
                filtered_state_covs[t-1]
            )
            filtered_state_means[t], filtered_state_covs[t] = self.filter_update(
                predicted_state_means[t],
                predicted_state_covs[t],
                observations[t]
        )
    return (
        filtered_state_means,
        filtered_state_covs,
        predicted_state_means,
        predicted_state_covs
    )
            
KalmanFilter.filter_predict = filter_predict
KalmanFilter.filter_update = filter_update
KalmanFilter.filter = filter

#### 平滑化
平滑化状態$\hat{\alpha_{t}} = E(\alpha_{t}|y)$、平滑化状態分散$V_{t} = Var(\alpha_{t}|y)$を求める。  
$r_{n}=0, N_{n}=0$を初期値として、$t=n, \dots, 1$について
$$
\begin{align*}
r_{t-1} &= Z_{t}'F_{t}^{-1}v_{t} + L_{t}'T_{t}r_{t}, & N_{t-1} &= Z_{t}'F_{t}^{-1}Z_{t} + L_{t}'T_{t}'N_{t}T_{t}L_{t}, \\
\hat{\alpha}_{t} &= a_{t|t} + P_{t|t}T_{t}'r_{t}, & V_{t} &= P_{t|t} - P_{t|t}T_{t}'N_{t}T_{t}P_{t|t} \\
\end{align*}
$$
ただし、$L_{t} = I_{m}-K_{t}Z_{t}$($I_{m}$は$m$次単位行列)

また、漸化式を用いずに$t=n, \dots, 1$について
$$
\begin{align*}
A_{t} &= P_{t|t}T_{t}'P_{t|t}^{-1} \\
\hat{\alpha}_{t} &= a_{t|t} + A_{t}(\hat{\alpha}_{t+1} - a_{t+1}) \\
V_{t} &= P_{t|t} + A_{t}(V_{t+1} - P_{t+1})A_{t}',
\end{align*}
$$

以上より
$$
\begin{align*}
\hat{\epsilon}_{t} &= v_{t} - Z_{t}P_{t}r_{t_1}, \\
\hat{\eta} &= Q_{t}R_{t}'r_{t} \\
Var(\epsilon |y) &= Z_{t}V_{t}Z_{t}' \\
Var(\eta | y) &= Q_{t} - Q_{t}R_{t}'N_{t}R_{t}Q_{t}
\end{align*}
$$

In [ ]:
def smooth_update(self,
                  filtered_state_mean,
                  filtered_state_cov,
                  predicted_state_mean,
                  predicted_state_cov,
                  next_smoothed_state_mean,
                  next_smoothed_state_cov
                  ):
    kalman_smoothing_gain = (
        filtered_state_cov
        @ self.system_matrix.T
        @ filtered_state_cov.T
    )
    
    smoothed_state_mean = (
        filtered_state_mean
        + kalman_smoothing_gain
        @ (next_smoothed_state_mean - predicted_state_mean)
    )
    smoothed_state_cov = (
        filtered_state_cov
        + kalman_smoothing_gain
        @ (next_smoothed_state_cov - predicted_state_cov)
        @ kalman_smoothing_gain.T
    )
    return (smoothed_state_mean, smoothed_state_cov)

def smooth(self, observations):
    (
        filtered_state_means,
        filtered_state_covs,
        predicted_state_means,
        predicted_state_covs
    ) = self.filter(observations)
    
    n_timesteps, n_dim_state = filtered_state_means.shape
    
    smoothed_state_means = np.zeros((n_timesteps, n_dim_state))
    smoothed_state_covs = np.zeros((n_timesteps, n_dim_state, n_dim_state))

    smoothed_state_means[-1] = filtered_state_means[-1]
    smoothed_state_covs[-1] = filtered_state_covs[-1]

    for t in reversed(range(n_timesteps - 1)):
        smoothed_state_means[t], smoothed_state_covs[t] = self.smooth_update(
            filtered_state_means[t],
            filtered_state_covs[t],
            predicted_state_means[t+1],
            predicted_state_covs[t+1],
            smoothed_state_means[t+1],
            smoothed_state_covs[t+1]
        )
    
    return (smoothed_state_means, smoothed_state_covs)

KalmanFilter.smooth_update = smooth_update
KalmanFilter.smooth = smooth



#### 対数尤度
初期分布について、$P_1$が有限で$a_1, P_1$ともに既知、さらに誤差分散行列$F_{t}$が正則行列である場合。

$$
\log{L} = -  \frac{np}{2} \log{2\pi} - \frac{1}{2} \sum_{t=1}^{n}(\log{|F_{t}|} + v_{t}'F_{t}^{-1}v_{t})
$$

ただし、$p$は観測値ベクトル$y_{1}, \dots, y_{n}$の次元。

### 散漫カルマンフィルタ (diffuse Kalman Filter)
時系列データ以外に初期状態に関する情報はないものと考える場合が多く、その場合、初期時点$t=1$はあくまで観測を始めた時点であり、状態遷移はそれ以前$t=0, -1, -2, \dots$からずっと続いてきたものだとすると、非定常的なモデルにおいては初期状態の分散は実質無限であると考えるべきである。その考えを反映したものがこのモデル。  
カルマンフィルタにおいて初期状態分散を$P_{1} \to \infty$としたもの(散漫初期化)

- - -

線形非ガウス状態空間モデルを以下のように定義する。  
$t=1, \dots, n$において

$$
\begin{align*}
p(y_{t}|\alpha_{1}, \dots , \alpha_{t}, y_{1}, \dots, y_{t-1}) &= p(y_{t}|Z_{t}\alpha_{t}) \\
\alpha_{t+1} &= T_{t}\alpha_{t} + R_{t}\eta_{t}, & \eta_{t} &\sim N(0, Q_{t}) \\
\alpha_{1} & \sim N(a_1, P_1)
\end{align*}
$$
ただし$p(y_{t}|Z_{t}\alpha_{t})$は信号$\theta_{t} = Z_{t}\alpha_{t}$をパラメータとする確率密度関数。  
また、各時点の観測の独立性を
$$
p(y|\alpha_{1}, \dots , \alpha_{n}) = \prod_{t=1}^{n}{p(y_{t}|\theta_{t})}
$$
として仮定する。さらに簡単のため、$y_{t} = (y_{1t}, \dots, y_{p, t})$が多変量時系列である場合には、各成分の独立性を
$$
p(y_{t}|\theta_{t}) = \prod_{j=1}^{p_{t}}{p(y_{jt})|\theta_{jt}}
$$
のように仮定する。観測値の各成分に対する確率分布$p(y_{jt})|\theta_{jt}$はそれぞれ異なる種類であっても良い。
- - -

1. 全観測値$y$が与えられた下での全時点の信号$\theta=(\theta_{1}', \dots, \theta_{n}')$の条件付きモード$\hat{\theta} = \text{argmax}_{\theta}p(\theta|p)$を求めて、そこから近似する線形ガウスモデルを導出する。
2. インポータンス・サンプリング(全観測値$y$が与えられた下での状態の条件付き分布からのシミュレーションサンプルを適切な重みを与えたガウス近似モデルから得る方法)を利用する。
3. シミュレーション・サンプルから求めた標本平均、標本分散やパーセンタイルを用いて、モデルの平滑化や予測、尤度を評価する。

- - -
非線形非ガウス状態空間モデルを以下のように定義する。  
$t=1, \dots, n$において
$$
\begin{align*}
y_{t} &\sim h_{t}(y_{t}|\alpha_{t}), \\
\alpha_{t+1} &\sim q_{t}(\alpha_{t+1}|\alpha_{t}), \\
\alpha_{1} &\sim p(\alpha_{1})
\end{align*}
$$
ただし、各時点$t=1, \dots, n$の$h_{t}(y_{t}|\alpha_{t})$および$q_{t}(\alpha_{t+1}|\alpha_{t}) $は条件付き確率密関数度あるいは条件付き確率関数、$p(\alpha_{1})$は初期状態の周辺確率関数あるいは周辺確率関数であり、それぞれが未知あるいは既知のモデルパラメータ$\psi$に依存するものとする。
- - -

### 粒子フィルタ  
各時点$t=1, \dots, n$に対して、与えられた$\alpha_{t}$のサンプルと重みを用いて翌期の状態$\alpha_{t+1}$に対するインポータンス密度を構成し、そこから$\alpha_{t+1}$のサンプルと重みを得ることを考える。これを初期地点から最終地点まで進めることによって、最終的に$t=1, \dots, n$についてフィルタ化状態分布$p(\alpha_{t}|Y_{t})$からの$\alpha_{t}$のサンプルを得ることができる。

### MCMC
事後分布からサンプリングする。